In [1]:
import pink
import pink.cpp as pinklib
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import figure
import skimage
import os
import glob

from skimage.measure import label, regionprops
from skimage.filters import gaussian
from skimage.filters import threshold_otsu

from skimage.morphology import skeletonize,skeletonize_3d
from skimage import img_as_int

from scipy import ndimage
from skimage.morphology import square
from skimage.morphology import disk
from skimage.morphology import dilation
from skimage.morphology import erosion
from skimage.morphology import remove_small_objects
from skimage import data, exposure, img_as_float
from skimage.exposure import histogram


def imview(image):

    plt.figure(figsize=(10,10))
    plt.imshow(pink.pink2numpy(image), cmap = "gray")
    plt.show()
    
    
def add_frame(img, nb):
    for i in range(len(img[0])):
        for k in range(nb):
            img[k][i] = 1
            img[len(img)-k-1][i] = 1
        
    for j in range(len(img)):
        for l in range(nb):
            img[j][l] = 1
            img[j][len(img[0])-l-1] = 1
            
    return img

def remove_frame(img, nb):
    for i in range(len(img[0])):
        for k in range(nb):
            img[k][i] = 0
            img[len(img)-k-1][i] = 0
        
    for j in range(len(img)):
        for l in range(nb):
            img[j][l] = 0
            img[j][len(img[0])-l-1] = 0
            
    return img


In [2]:
Images_GC_moins = np.sort(glob.glob("/nfs/jupyter/boviom/Projet_Morpho/GC-/CK10/*.tif"))
Images_GC_plus = np.sort(glob.glob("/nfs/jupyter/boviom/Projet_Morpho/GC+/CK10/*.tif"))
Images_GC_placebo = np.sort(glob.glob("/nfs/jupyter/boviom/Projet_Morpho/GC+placebo/CK10/*.tif"))
Images_GC_creme = np.sort(glob.glob("/nfs/jupyter/boviom/Projet_Morpho/GC+creme/CK10/*.tif"))


In [3]:
Epais1 = list()
Epais2 = list()
Epais3 = list()
Invag = list()
data_GC_moins = list()

for i in Images_GC_moins:
    img = pinklib.readimage(i)
#     imview(img)
    img_CK_eros = pinklib.erosion(img, pinklib.genball(25))
#     imview(img_CK_eros)
    
    test = pinklib.geodilat(img_CK_eros,img,8)
#     imview(test)
    
    valeur_h = int(np.min(pink.pink2numpy(test)) + ((np.max(pink.pink2numpy(test)) - np.min(pink.pink2numpy(test)))/2))
    img_CK_eros_clean = pinklib.heightminima(test, 4, valeur_h)
#     imview(img_CK_eros_clean)
    
    valeur_thd = int(np.min(pink.pink2numpy(img_CK_eros_clean)) + ((np.max(pink.pink2numpy(img_CK_eros_clean)) - np.min(pink.pink2numpy(img_CK_eros_clean)))/2))
    thd = pinklib.seuil(img_CK_eros_clean, valeur_thd)
#     imview(thd)

    thd_np = pink.pink2numpy(thd)
    img_gaussian = skimage.filters.gaussian(thd_np, sigma=30)
    thresh = threshold_otsu(img_gaussian)
    binary = img_gaussian > thresh
#     plt.imshow(binary)
#     plt.show()

    skeleton = skeletonize(binary)
    framed_skeleton = add_frame(skeleton, 50)

    main_skeleton = pinklib.skeleton(pink.numpy2pink(framed_skeleton.astype('uint8')), 0, 8, -1)
    main_skeleton_clean = remove_frame(pink.pink2numpy(main_skeleton).astype(int), 50)
    
    branches = main_skeleton_clean - remove_frame(skeleton, 50)
    branches_pink = pink.numpy2pink(branches.astype('uint8'))
    
#     imview(pinklib.dilatball(pink.numpy2pink(remove_frame(skeleton, 50).astype('uint8')), 1))
#     imview(pinklib.dilatball(pink.numpy2pink(main_skeleton_clean.astype('uint8')), 1))
#     imview(pinklib.dilatball(branches_pink, 1))
    
    
    unframed_skeleton = remove_frame(skeleton, 50)
    binary_pink = pink.numpy2pink(binary.astype('uint8'))
    distance_binary = pinklib.distc(binary_pink,0)
    
    
    main_dilated = pink.pink2numpy(pinklib.dilatball(pink.numpy2pink(main_skeleton_clean.astype('uint8')), 3)).astype(int)
    
    ### methode 1
    epaisseur1 = main_skeleton_clean * remove_frame(pink.pink2numpy(distance_binary).astype(int), 50)
    Epaisseur_moyenne_1 = sum(sum(epaisseur1)) / (sum(sum(main_dilated)) / (7*255))

    ### methode 2
    epaisseur2 = unframed_skeleton * remove_frame(pink.pink2numpy(distance_binary).astype(int), 50)
    Epaisseur_moyenne_2 = sum(sum(epaisseur2)) / (sum(sum(main_dilated)) / (7*255))

    ### methode 3
    binary_borderless = remove_frame(binary, 50)
    Epaisseur_moyenne_3 = sum(sum(binary_borderless)) / (sum(sum(main_dilated)) / (7*255))

    
    voisins = pink.pink2numpy(pinklib.nbvois(pink.numpy2pink(unframed_skeleton.astype('uint8')), 8)).astype(int)

    points_triples = np.where(voisins==3)

    branches_complet = pink.pink2numpy(branches_pink).astype(int)


    for i in range (len(points_triples[0])):
        branches_complet[points_triples[0][i],points_triples[1][i]] = 255

    branches_complet_labelise, nb_branches = skimage.measure.label(branches_complet, return_num = True)
    branches_props = regionprops(branches_complet_labelise)

    list_of_area = []
    for i in range(nb_branches):
        list_of_area.append(branches_props[i].area)
        
    indice = 0
    if len(list_of_area) != 0:
        indice_branche_max = list_of_area.index(max(list_of_area))

        
        for i in range(min(len(points_triples[0]), nb_branches)):
            if branches_complet_labelise[points_triples[0][i],points_triples[1][i]] == indice_branche_max + 1:
                indice = i

    if indice != 0:
        dist = remove_frame(pink.pink2numpy(distance_binary).astype(int), 50)

        taille_branche =  max(list_of_area) +  (dist[points_triples[0][indice],points_triples[1][indice]]*2)

        degres_invagination = taille_branche/Epaisseur_moyenne_3
    else:
        degres_invagination = float(0)
    
    
    if Epaisseur_moyenne_3 < 10000:
        
        Epais1.append(Epaisseur_moyenne_1)
        Epais2.append(Epaisseur_moyenne_2)
        Epais3.append(Epaisseur_moyenne_3)
        Invag.append(degres_invagination)
        
    else:
    
        Epais1.append(np.nan)
        Epais2.append(np.nan)
        Epais3.append(np.nan)
        Invag.append(np.nan)
        
data_GC_moins = [Images_GC_moins, Epais1, Epais2, Epais3, Invag]


data_GC_moins = [Epais1, Epais2, Epais3, Invag]
np.savetxt("GC_moins.csv", data_GC_moins, delimiter=";")

In [4]:
Epais1 = list()
Epais2 = list()
Epais3 = list()
Invag = list()
data_GC_plus = list()

for i in Images_GC_plus:
    img = pinklib.readimage(i)
#     imview(img)
    img_CK_eros = pinklib.erosion(img, pinklib.genball(25))
#     imview(img_CK_eros)
    
    test = pinklib.geodilat(img_CK_eros,img,8)
#     imview(test)
    
    valeur_h = int(np.min(pink.pink2numpy(test)) + ((np.max(pink.pink2numpy(test)) - np.min(pink.pink2numpy(test)))/2))
    img_CK_eros_clean = pinklib.heightminima(test, 4, valeur_h)
#     imview(img_CK_eros_clean)
    
    valeur_thd = int(np.min(pink.pink2numpy(img_CK_eros_clean)) + ((np.max(pink.pink2numpy(img_CK_eros_clean)) - np.min(pink.pink2numpy(img_CK_eros_clean)))/2))
    thd = pinklib.seuil(img_CK_eros_clean, valeur_thd)
#     imview(thd)

    thd_np = pink.pink2numpy(thd)
    img_gaussian = skimage.filters.gaussian(thd_np, sigma=30)
    thresh = threshold_otsu(img_gaussian)
    binary = img_gaussian > thresh
#     plt.imshow(binary)
#     plt.show()

    skeleton = skeletonize(binary)
    framed_skeleton = add_frame(skeleton, 50)

    main_skeleton = pinklib.skeleton(pink.numpy2pink(framed_skeleton.astype('uint8')), 0, 8, -1)
    main_skeleton_clean = remove_frame(pink.pink2numpy(main_skeleton).astype(int), 50)
    
    branches = main_skeleton_clean - remove_frame(skeleton, 50)
    branches_pink = pink.numpy2pink(branches.astype('uint8'))
    
#     imview(pinklib.dilatball(pink.numpy2pink(remove_frame(skeleton, 50).astype('uint8')), 1))
#     imview(pinklib.dilatball(pink.numpy2pink(main_skeleton_clean.astype('uint8')), 1))
#     imview(pinklib.dilatball(branches_pink, 1))
    
    
    unframed_skeleton = remove_frame(skeleton, 50)
    binary_pink = pink.numpy2pink(binary.astype('uint8'))
    distance_binary = pinklib.distc(binary_pink,0)
    
    
    main_dilated = pink.pink2numpy(pinklib.dilatball(pink.numpy2pink(main_skeleton_clean.astype('uint8')), 3)).astype(int)
    
    ### methode 1
    epaisseur1 = main_skeleton_clean * remove_frame(pink.pink2numpy(distance_binary).astype(int), 50)
    Epaisseur_moyenne_1 = sum(sum(epaisseur1)) / (sum(sum(main_dilated)) / (7*255))

    ### methode 2
    epaisseur2 = unframed_skeleton * remove_frame(pink.pink2numpy(distance_binary).astype(int), 50)
    Epaisseur_moyenne_2 = sum(sum(epaisseur2)) / (sum(sum(main_dilated)) / (7*255))

    ### methode 3
    binary_borderless = remove_frame(binary, 50)
    Epaisseur_moyenne_3 = sum(sum(binary_borderless)) / (sum(sum(main_dilated)) / (7*255))

    
    voisins = pink.pink2numpy(pinklib.nbvois(pink.numpy2pink(unframed_skeleton.astype('uint8')), 8)).astype(int)

    points_triples = np.where(voisins==3)

    branches_complet = pink.pink2numpy(branches_pink).astype(int)


    for i in range (len(points_triples[0])):
        branches_complet[points_triples[0][i],points_triples[1][i]] = 255

    branches_complet_labelise, nb_branches = skimage.measure.label(branches_complet, return_num = True)
    branches_props = regionprops(branches_complet_labelise)

    list_of_area = []
    for i in range(nb_branches):
        list_of_area.append(branches_props[i].area)
        
    indice = 0
    if len(list_of_area) != 0:
        indice_branche_max = list_of_area.index(max(list_of_area))

        
        for i in range(min(len(points_triples[0]), nb_branches)):
            if branches_complet_labelise[points_triples[0][i],points_triples[1][i]] == indice_branche_max + 1:
                indice = i

    if indice != 0:
        dist = remove_frame(pink.pink2numpy(distance_binary).astype(int), 50)

        taille_branche =  max(list_of_area) +  (dist[points_triples[0][indice],points_triples[1][indice]]*2)

        degres_invagination = taille_branche/Epaisseur_moyenne_3
    else:
        degres_invagination = float(0)
    
    
    if Epaisseur_moyenne_3 < 10000:
        
        Epais1.append(Epaisseur_moyenne_1)
        Epais2.append(Epaisseur_moyenne_2)
        Epais3.append(Epaisseur_moyenne_3)
        Invag.append(degres_invagination)
        
    else:
    
        Epais1.append(np.nan)
        Epais2.append(np.nan)
        Epais3.append(np.nan)
        Invag.append(np.nan)
        
data_GC_plus = [Epais1, Epais2, Epais3, Invag]

np.savetxt("GC_plus.csv", data_GC_plus, delimiter=";")

In [5]:
print(data_GC_plus)

[[74.451107077504972, 81.393108491099824, nan, 60.761038056004907, 66.997454707190073, nan, 68.193737278299764, 69.32471359239328, nan, nan, 69.373339499786667, 68.757327350467037, 65.92965686831981, nan, 60.659032455069614], [94.711384092235974, 84.424561449816537, nan, 60.996212062714996, 77.913187584361381, nan, 74.397494444736054, 83.145036151738267, nan, nan, 69.416388815689842, 73.328701301102086, 70.619677933511682, nan, 60.659032455069614], [183.78880164047706, 176.54303872574113, nan, 144.84134483597347, 155.52011160554846, nan, 161.39733114038924, 175.32495295321735, nan, nan, 154.09502627540428, 155.67781796113331, 158.84079333892041, nan, 151.08378292102756], [0.0, 1.659651958609222, nan, 0.0, 2.5141442863139662, nan, 0.0, 0.0, nan, nan, 0.0, 0.0, 2.4426974446804732, nan, 0.0]]


In [6]:
Epais1 = list()
Epais2 = list()
Epais3 = list()
Invag = list()
data_GC_placebo = list()

for i in Images_GC_plus:
    img = pinklib.readimage(i)
#     imview(img)
    img_CK_eros = pinklib.erosion(img, pinklib.genball(25))
#     imview(img_CK_eros)
    
    test = pinklib.geodilat(img_CK_eros,img,8)
#     imview(test)
    
    valeur_h = int(np.min(pink.pink2numpy(test)) + ((np.max(pink.pink2numpy(test)) - np.min(pink.pink2numpy(test)))/2))
    img_CK_eros_clean = pinklib.heightminima(test, 4, valeur_h)
#     imview(img_CK_eros_clean)
    
    valeur_thd = int(np.min(pink.pink2numpy(img_CK_eros_clean)) + ((np.max(pink.pink2numpy(img_CK_eros_clean)) - np.min(pink.pink2numpy(img_CK_eros_clean)))/2))
    thd = pinklib.seuil(img_CK_eros_clean, valeur_thd)
#     imview(thd)

    thd_np = pink.pink2numpy(thd)
    img_gaussian = skimage.filters.gaussian(thd_np, sigma=30)
    thresh = threshold_otsu(img_gaussian)
    binary = img_gaussian > thresh
#     plt.imshow(binary)
#     plt.show()

    skeleton = skeletonize(binary)
    framed_skeleton = add_frame(skeleton, 50)

    main_skeleton = pinklib.skeleton(pink.numpy2pink(framed_skeleton.astype('uint8')), 0, 8, -1)
    main_skeleton_clean = remove_frame(pink.pink2numpy(main_skeleton).astype(int), 50)
    
    branches = main_skeleton_clean - remove_frame(skeleton, 50)
    branches_pink = pink.numpy2pink(branches.astype('uint8'))
    
#     imview(pinklib.dilatball(pink.numpy2pink(remove_frame(skeleton, 50).astype('uint8')), 1))
#     imview(pinklib.dilatball(pink.numpy2pink(main_skeleton_clean.astype('uint8')), 1))
#     imview(pinklib.dilatball(branches_pink, 1))
    
    
    unframed_skeleton = remove_frame(skeleton, 50)
    binary_pink = pink.numpy2pink(binary.astype('uint8'))
    distance_binary = pinklib.distc(binary_pink,0)
    
    
    main_dilated = pink.pink2numpy(pinklib.dilatball(pink.numpy2pink(main_skeleton_clean.astype('uint8')), 3)).astype(int)
    
    ### methode 1
    epaisseur1 = main_skeleton_clean * remove_frame(pink.pink2numpy(distance_binary).astype(int), 50)
    Epaisseur_moyenne_1 = sum(sum(epaisseur1)) / (sum(sum(main_dilated)) / (7*255))

    ### methode 2
    epaisseur2 = unframed_skeleton * remove_frame(pink.pink2numpy(distance_binary).astype(int), 50)
    Epaisseur_moyenne_2 = sum(sum(epaisseur2)) / (sum(sum(main_dilated)) / (7*255))

    ### methode 3
    binary_borderless = remove_frame(binary, 50)
    Epaisseur_moyenne_3 = sum(sum(binary_borderless)) / (sum(sum(main_dilated)) / (7*255))

    
    voisins = pink.pink2numpy(pinklib.nbvois(pink.numpy2pink(unframed_skeleton.astype('uint8')), 8)).astype(int)

    points_triples = np.where(voisins==3)

    branches_complet = pink.pink2numpy(branches_pink).astype(int)


    for i in range (len(points_triples[0])):
        branches_complet[points_triples[0][i],points_triples[1][i]] = 255

    branches_complet_labelise, nb_branches = skimage.measure.label(branches_complet, return_num = True)
    branches_props = regionprops(branches_complet_labelise)

    list_of_area = []
    for i in range(nb_branches):
        list_of_area.append(branches_props[i].area)
        
    indice = 0
    if len(list_of_area) != 0:
        indice_branche_max = list_of_area.index(max(list_of_area))

        for i in range(min(len(points_triples[0]), nb_branches)):
            if branches_complet_labelise[points_triples[0][i],points_triples[1][i]] == indice_branche_max + 1:
                indice = i

    if indice != 0:
        dist = remove_frame(pink.pink2numpy(distance_binary).astype(int), 50)

        taille_branche =  max(list_of_area) +  (dist[points_triples[0][indice],points_triples[1][indice]]*2)

        degres_invagination = taille_branche/Epaisseur_moyenne_3
    else:
        degres_invagination = float(0)
    
    
    if Epaisseur_moyenne_3 < 10000:
        
        Epais1.append(Epaisseur_moyenne_1)
        Epais2.append(Epaisseur_moyenne_2)
        Epais3.append(Epaisseur_moyenne_3)
        Invag.append(degres_invagination)
        
    else:
    
        Epais1.append(np.nan)
        Epais2.append(np.nan)
        Epais3.append(np.nan)
        Invag.append(np.nan)
        
data_GC_placebo = [Epais1, Epais2, Epais3, Invag]
np.savetxt("GC_placebo.csv", data_GC_placebo, delimiter=";")

In [7]:
Epais1 = list()
Epais2 = list()
Epais3 = list()
Invag = list()
data_GC_creme = list()

for i in Images_GC_creme:
    img = pinklib.readimage(i)
#     imview(img)
    img_CK_eros = pinklib.erosion(img, pinklib.genball(25))
#     imview(img_CK_eros)
    
    test = pinklib.geodilat(img_CK_eros,img,8)
#     imview(test)
    
    valeur_h = int(np.min(pink.pink2numpy(test)) + ((np.max(pink.pink2numpy(test)) - np.min(pink.pink2numpy(test)))/2))
    img_CK_eros_clean = pinklib.heightminima(test, 4, valeur_h)
#     imview(img_CK_eros_clean)
    
    valeur_thd = int(np.min(pink.pink2numpy(img_CK_eros_clean)) + ((np.max(pink.pink2numpy(img_CK_eros_clean)) - np.min(pink.pink2numpy(img_CK_eros_clean)))/2))
    thd = pinklib.seuil(img_CK_eros_clean, valeur_thd)
#     imview(thd)

    thd_np = pink.pink2numpy(thd)
    img_gaussian = skimage.filters.gaussian(thd_np, sigma=30)
    thresh = threshold_otsu(img_gaussian)
    binary = img_gaussian > thresh
#     plt.imshow(binary)
#     plt.show()

    skeleton = skeletonize(binary)
    framed_skeleton = add_frame(skeleton, 50)

    main_skeleton = pinklib.skeleton(pink.numpy2pink(framed_skeleton.astype('uint8')), 0, 8, -1)
    main_skeleton_clean = remove_frame(pink.pink2numpy(main_skeleton).astype(int), 50)
    
    branches = main_skeleton_clean - remove_frame(skeleton, 50)
    branches_pink = pink.numpy2pink(branches.astype('uint8'))
    
#     imview(pinklib.dilatball(pink.numpy2pink(remove_frame(skeleton, 50).astype('uint8')), 1))
#     imview(pinklib.dilatball(pink.numpy2pink(main_skeleton_clean.astype('uint8')), 1))
#     imview(pinklib.dilatball(branches_pink, 1))
    
    
    unframed_skeleton = remove_frame(skeleton, 50)
    binary_pink = pink.numpy2pink(binary.astype('uint8'))
    distance_binary = pinklib.distc(binary_pink,0)
    
    
    main_dilated = pink.pink2numpy(pinklib.dilatball(pink.numpy2pink(main_skeleton_clean.astype('uint8')), 3)).astype(int)
    
    ### methode 1
    epaisseur1 = main_skeleton_clean * remove_frame(pink.pink2numpy(distance_binary).astype(int), 50)
    Epaisseur_moyenne_1 = sum(sum(epaisseur1)) / (sum(sum(main_dilated)) / (7*255))

    ### methode 2
    epaisseur2 = unframed_skeleton * remove_frame(pink.pink2numpy(distance_binary).astype(int), 50)
    Epaisseur_moyenne_2 = sum(sum(epaisseur2)) / (sum(sum(main_dilated)) / (7*255))

    ### methode 3
    binary_borderless = remove_frame(binary, 50)
    Epaisseur_moyenne_3 = sum(sum(binary_borderless)) / (sum(sum(main_dilated)) / (7*255))

    
    voisins = pink.pink2numpy(pinklib.nbvois(pink.numpy2pink(unframed_skeleton.astype('uint8')), 8)).astype(int)

    points_triples = np.where(voisins==3)

    branches_complet = pink.pink2numpy(branches_pink).astype(int)


    for i in range (len(points_triples[0])):
        branches_complet[points_triples[0][i],points_triples[1][i]] = 255

    branches_complet_labelise, nb_branches = skimage.measure.label(branches_complet, return_num = True)
    branches_props = regionprops(branches_complet_labelise)

    list_of_area = []
    for i in range(nb_branches):
        list_of_area.append(branches_props[i].area)
        
    indice = 0
    if len(list_of_area) != 0:
        indice_branche_max = list_of_area.index(max(list_of_area))
        
        for i in range(min(len(points_triples[0]), nb_branches)):
            if branches_complet_labelise[points_triples[0][i],points_triples[1][i]] == indice_branche_max + 1:
                indice = i

    if indice != 0:
        dist = remove_frame(pink.pink2numpy(distance_binary).astype(int), 50)

        taille_branche =  max(list_of_area) +  (dist[points_triples[0][indice],points_triples[1][indice]]*2)

        degres_invagination = taille_branche/Epaisseur_moyenne_3
    else:
        degres_invagination = float(0)
    
    
    if Epaisseur_moyenne_3 < 10000:
        
        Epais1.append(Epaisseur_moyenne_1)
        Epais2.append(Epaisseur_moyenne_2)
        Epais3.append(Epaisseur_moyenne_3)
        Invag.append(degres_invagination)
        
    else:
    
        Epais1.append(np.nan)
        Epais2.append(np.nan)
        Epais3.append(np.nan)
        Invag.append(np.nan)
        
data_GC_creme = [Epais1, Epais2, Epais3, Invag]
np.savetxt("GC_creme.csv", data_GC_creme, delimiter=";")